In [1]:
#注意事项:
#当运行本Notebook的程序后，如果要关闭Notebook，请选择菜单: File > Close and Halt 才能确实停止当前正在运行的程序，并且释放资源
#如果没有使用以上方法，只关闭此分页，程序仍在运行，未释放资源，当您打开并运行其他的Notebook，可能会发生错误

# 12.7	如何进行数据准备?

In [2]:
sc.master

u'local[*]'

In [3]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonsparkexample/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"
#如果要在cluster模式运行(hadoop yarn 或Spark Stand alone)，请按照书上的说明，先把文件上传到HDFS目录

In [4]:
rawUserData = sc.textFile(Path+"data/u.data")
rawUserData.count()

100000

In [5]:
rawUserData.first()

u'196\t242\t3\t881250949'

In [6]:
print rawUserData.first()

196	242	3	881250949


In [7]:
rawUserData.take(5)

[u'196\t242\t3\t881250949',
 u'186\t302\t3\t891717742',
 u'22\t377\t1\t878887116',
 u'244\t51\t2\t880606923',
 u'166\t346\t1\t886397596']

In [8]:
for x in rawUserData.take(5): 
    print x

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596


In [9]:
from pyspark.mllib.recommendation import Rating

In [10]:
rawRatings = rawUserData.map(lambda line: line.split("\t")[:3] )
rawRatings.take(5)

[[u'196', u'242', u'3'],
 [u'186', u'302', u'3'],
 [u'22', u'377', u'1'],
 [u'244', u'51', u'2'],
 [u'166', u'346', u'1']]

In [11]:
ratingsRDD = rawRatings.map(lambda x: (x[0],x[1],x[2]))
ratingsRDD .take(5)

[(u'196', u'242', u'3'),
 (u'186', u'302', u'3'),
 (u'22', u'377', u'1'),
 (u'244', u'51', u'2'),
 (u'166', u'346', u'1')]

In [12]:
numRatings = ratingsRDD.count()
numRatings

100000

In [13]:
numUsers = ratingsRDD.map(lambda x: x[0] ).distinct().count()
numUsers 

943

In [14]:
numMovies = ratingsRDD.map(lambda x: x[1]).distinct().count() 
numMovies

1682

In [15]:
ratingsRDD.persist()

PythonRDD[20] at RDD at PythonRDD.scala:48

# 12.8	如何训练模型? 

In [16]:
from pyspark.mllib.recommendation import ALS

In [17]:
model = ALS.train(ratingsRDD, 10, 10, 0.01)
print model

In [18]:
print model

# 12.9	如何使用模型进行推荐?

In [19]:
model.recommendProducts(100,5)

[Rating(user=100, product=394, rating=5.85837125113027),
 Rating(user=100, product=1426, rating=5.826015175447894),
 Rating(user=100, product=1268, rating=5.81984582554753),
 Rating(user=100, product=1664, rating=5.808145526882063),
 Rating(user=100, product=1169, rating=5.757741851342477)]

In [20]:
model.predict(100, 1141)

3.100628178085713

In [21]:
model.recommendUsers(product=200,num=5)

[Rating(user=34, product=200, rating=7.013932671740623),
 Rating(user=909, product=200, rating=6.853612463673836),
 Rating(user=444, product=200, rating=6.198114805350716),
 Rating(user=143, product=200, rating=6.173815075525909),
 Rating(user=196, product=200, rating=6.1088936081610345)]

# 12.10	显示推荐的电影的名称

In [22]:
itemRDD = sc.textFile(Path+"data/u.item")
itemRDD.count()

1682

In [23]:
movieTitle= itemRDD.map( lambda line : line.split("|"))     \
                                   .map(lambda a: (float(a[0]),a[1]))       \
                                   .collectAsMap()
len(movieTitle)

1682

In [24]:
movieTitle.items()[:5]

[(1.0, u'Toy Story (1995)'),
 (2.0, u'GoldenEye (1995)'),
 (3.0, u'Four Rooms (1995)'),
 (4.0, u'Get Shorty (1995)'),
 (5.0, u'Copycat (1995)')]

In [25]:
for i in range(1,6): 
    print str(i)+":"+movieTitle[i]

1:Toy Story (1995)
2:GoldenEye (1995)
3:Four Rooms (1995)
4:Get Shorty (1995)
5:Copycat (1995)


In [26]:
movieTitle[5]

u'Copycat (1995)'

In [27]:
recommendP= model.recommendProducts(100,5) 
for p in recommendP:
    print  "对用户"+ str(p[0]) +                    \
               "推荐电影"+ str(movieTitle[p[1]]) + \
               "推荐评分"+ str(p[2])         

针对用户100推荐电影Radioland Murders (1994)推荐评分5.85837125113
针对用户100推荐电影Grace of My Heart (1996)推荐评分5.82601517545
针对用户100推荐电影Bitter Moon (1992)推荐评分5.81984582555
针对用户100推荐电影8 Heads in a Duffel Bag (1997)推荐评分5.80814552688
针对用户100推荐电影Fresh (1994)推荐评分5.75774185134
